In [1]:
import pandas as pd
from klse_scrapper import *
from plot_utils import *
from utils import *
from fastai.imports import *
from fastai.structured import *
from loess.loess_1d import loess_1d
from datetime import datetime, date, time, timedelta
from workalendar.asia import Malaysia
from counter import Counter
import os
from session_handler import *
from shareinvestor_scrapper import *

C:\ProgramData\Anaconda3\envs\fastai\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning:

The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.



In [2]:
topglov=Counter('SUPERMX','7106',scheduler_on=False)
topglov.set_timestamp(datetime.combine(date.today()-timedelta(days=2), time(9)).timestamp())
topglov.get_day_volume()
topglov.detect_shark()

Health Care
False


In [29]:
df_=topglov.buffer_temp.copy()
df_[['last_done_vol','last_done_price','type']]=df_[['last_done_vol','last_done_price','type']].fillna(method='ffill')
df_['last_done_price']=df_['last_done_price'].fillna(topglov.today_open)
df_['last_done_vol']=df_['last_done_vol'].fillna(0)
df_['buy_queue_price_diff']=df_['buy_queue_price']-df_['last_done_price']
df_['sell_queue_price_diff']=df_['sell_queue_price']-df_['last_done_price']
df_['buy_queue_type']=df_['buy_queue_price_diff'].dropna().transform(lambda x: 'Buy Up' if (x > 0) else 'Buy Down')
df_['sell_queue_type']=df_['sell_queue_price_diff'].dropna().transform(lambda x: 'Sell Up' if (x > 0) else 'Sell Down')
df_['buy_queue_price'][df_['buy_queue_price'].isna()]=df_['sell_queue_price'][df_['sell_queue_price'].notna()]
#df_['buy_queue_type'][df_['buy_queue_type'].isna()]=df_['sell_queue_type'][df_['sell_queue_type'].notna()]
#df_['buy_vol_chg']=df_['sell_vol_chg'][df_['sell_vol_chg']>0]
#df_=df_.rename(columns={'buy_queue_price':'queue_price','buy_queue_type':'queue_type','buy_vol_chg':'vol_chg'})
df_=df_.drop(columns=['sell_queue_price_diff','buy_queue_price_diff'])
#df__=df_[['time','queue_price','vol_chg','queue_type','last_done_price']].dropna().copy()
df_

C:\ProgramData\Anaconda3\envs\fastai\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,time,buy_queue_vol,buy_queue_price,last_done_vol,last_done_price,type,sell_queue_vol,sell_queue_price,buy_vol_chg,sell_vol_chg,buy_queue_type,sell_queue_type
0,2020-06-19 06:05:14,66.0,6.93,0.0,6.93,NaN,NaN,NaN,NaN,NaN,Buy Down,NaN
1,2020-06-19 06:05:14,NaN,6.98,0.0,6.93,NaN,30.0,6.98,NaN,NaN,NaN,Sell Up
2,2020-06-19 08:30:00,NaN,6.75,0.0,6.93,NaN,14.0,6.75,NaN,NaN,NaN,Sell Down
3,2020-06-19 08:30:00,11.0,6.98,0.0,6.93,NaN,NaN,NaN,NaN,NaN,Buy Up,NaN
4,2020-06-19 08:30:01,16.0,6.98,0.0,6.93,NaN,NaN,NaN,5.0,NaN,Buy Up,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
45658,2020-06-19 16:59:47,NaN,NaN,4.0,7.40,Sell Down,NaN,NaN,NaN,NaN,NaN,NaN
45659,2020-06-19 16:59:49,3586.0,7.40,4.0,7.40,Sell Down,NaN,NaN,100.0,NaN,Buy Down,NaN
45660,2020-06-19 16:59:58,3581.0,7.40,4.0,7.40,Sell Down,NaN,NaN,-5.0,NaN,Buy Down,NaN
45661,2020-06-19 16:59:58,NaN,NaN,5.0,7.40,Sell Down,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
interval='1min'
df_=df_.set_index('time')
#df_['last_done_price']=df_['last_done_price'].fillna(method='ffill').fillna(topglov.today_open)
#df_['']=df_.resample(interval).agg({'last_done_price':'last'}).dropna().pct_change()
df__=pd.DataFrame()
df__=df__.append(topglov.sharks.set_index('time').resample(interval).agg({'buy_vol_chg':'count'}))
df__=df__.rename(columns={'buy_vol_chg':'buy_shark'})
df__['buy_queue_type']=df_.resample(interval).agg({'buy_queue_type':'last'})
df__['last_done_price_diff']=df_.resample(interval).agg({'last_done_price':'last'}).dropna().diff()
df__['last_done_type']=df_.resample(interval).agg({'type':'last'})
df__['last_done_price_pct']=df_.resample(interval).agg({'last_done_price':'last'}).dropna().pct_change()
df__['last_done_vol_sum']=df_.resample(interval).agg({'last_done_vol':'sum'}).cumsum()
df__['last_done_price_pct']=round(df__['last_done_price_pct']*100,2)
df__['last_done_price']=df_.resample(interval).agg({'last_done_price':'last'})
df__['last_open_price_diff']=df__['last_done_price']-topglov.today_open
df__['last_open_price_pct']=df__['last_done_price'].transform(lambda x: round((x - topglov.today_open)/topglov.today_open * 100,2))
df__=df__.fillna(method='ffill')

In [31]:
df__

,buy_shark,buy_queue_type,last_done_price_diff,last_done_type,last_done_price_pct,last_done_vol_sum,last_done_price,last_open_price_diff,last_open_price_pct
time,,,,,,,,,
2020-06-19 09:05:00,1,Buy Down,0.00,Buy Up,0.00,36933.0,6.88,-0.05,-0.72
2020-06-19 09:06:00,1,Buy Down,0.02,Buy Up,0.29,41272.0,6.90,-0.03,-0.43
2020-06-19 09:07:00,0,Buy Down,0.04,Sell Down,0.58,49919.0,6.94,0.01,0.14
2020-06-19 09:08:00,0,Buy Down,0.07,Buy Up,1.01,58749.0,7.01,0.08,1.15
2020-06-19 09:09:00,0,Buy Down,-0.06,Buy Up,-0.86,65167.0,6.95,0.02,0.29
...,...,...,...,...,...,...,...,...,...
2020-06-19 16:48:00,0,Buy Up,0.00,Buy Up,0.00,947201.0,7.47,0.54,7.79
2020-06-19 16:49:00,0,Buy Up,0.00,Buy Up,0.00,947265.0,7.47,0.54,7.79
2020-06-19 16:50:00,0,Buy Down,-0.07,Sell Down,-0.94,971009.0,7.40,0.47,6.78
